# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
#Include api directory in path
if module_path not in sys.path:
    sys.path.append(module_path + "\\api")

# Import API key
from api_keys import g_key

output_data_file = "../output_data/"



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
print(output_data_file + "cities.csv")
df = pd.read_csv(output_data_file + "cities.csv")
df.head()

#Convert Object Types to String
df["City"] = df["City"].astype("string") 
df["Country"] = df["Country"].astype("string")

#Drop index column saved to csv
df = df.iloc[:,1:]
df.head()



../output_data/cities.csv


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kodiak,57.7900,-152.4072,30.20,64,1,6.91,US,1612148328
1,muros,42.7762,-9.0603,57.99,84,88,8.01,ES,1612148599
2,cayenne,4.9333,-52.3333,80.60,78,75,10.36,GF,1612148411
3,ushuaia,-54.8000,-68.3000,50.00,71,75,20.71,AR,1612148490
4,cape town,-33.9258,18.4232,64.00,72,20,21.85,ZA,1612148476


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
#Setup Ideal Weather Conditions Parameters
min_temp = 70
max_temp = 82
min_windspeed = 0
max_windspeed = 15
min_cloudiness = 0
max_cloudiness = 20
max_humidity = 80

perfect_conditions_df = df.loc[df["Max Temp"].between(min_temp,max_temp) &
                               df["Cloudiness"].between(min_cloudiness,max_cloudiness) &
                               df["Wind Speed"].between(min_windspeed,max_windspeed) &
                               df["Humidity"] < max_humid]

perfect_conditions_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,lokoja,7.8024,6.7430,74.35,52,0,3.27,NG,1612148600
7,saint-philippe,-21.3585,55.7679,80.60,61,0,10.36,RE,1612148329
8,grand gaube,-20.0064,57.6608,80.01,80,8,4.00,MU,1612148601
11,busselton,-33.6500,115.3333,82.00,43,0,7.00,AU,1612148602
15,georgetown,-7.9333,-14.4167,76.71,84,0,12.88,SH,1612148603
40,aloleng,16.1307,119.7824,80.35,56,0,8.16,PH,1612148609
42,ahipara,-35.1667,173.1667,71.02,56,2,7.52,NZ,1612148358
78,pisco,-13.7000,-76.2167,70.00,73,0,10.36,PE,1612148360
83,hilo,19.7297,-155.0900,77.00,61,1,8.05,US,1612148599
102,sao filipe,14.8961,-24.4956,71.78,74,0,5.44,CV,1612148625


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
